In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [5]:
conn = sqlite3.connect('C:\\Users\\jfran\\OneDrive\\Documents\\Baseball Database\\MLB_20002020_database.sqlite')

In [66]:
## Get the max of the score in all innings

runsPerInning = pd.read_sql_query("""
SELECT gameKey, teamAtBat, inn, MAX(scoreTeamAtBat) AS maxScoreTeamAtBat
FROM "gamelogs.playbyplay"
--WHERE year = 2014
GROUP BY gameKey, teamAtBat, inn
; """, conn)

In [74]:
# runsPerInning.sample(20)

In [75]:
## Get all at-bats, 
## Need to know the situation, that is the # of outs and the runners on bases.
## The objective is to calculate the number of runs in that inning following a situation until the end of the inning

atBatsPerInning = pd.read_sql_query("""
SELECT gameKey, teamAtBat, inn, out, runnerOnBase, scoreTeamAtBat
FROM "gamelogs.playbyplay"
--WHERE year = 2019
GROUP BY gameKey, teamAtBat, inn, out, runnerOnBase
; """, conn)

In [76]:
## merge the datasets
## Then compute the runs created following every situation

df = atBatsPerInning.merge(runsPerInning, on = ['gameKey', 'teamAtBat', 'inn'])
df['runsScoredAfter'] = df['maxScoreTeamAtBat'] - df['scoreTeamAtBat']

In [77]:
# atBatsPerInning.shape

In [78]:
# atBatsPerInning[(atBatsPerInning['gameKey']=='20140503BOSOAK0') & (atBatsPerInning['runnerOnBase']=='123')]

In [79]:
df.sample(20)

,gameKey,teamAtBat,inn,out,runnerOnBase,scoreTeamAtBat,maxScoreTeamAtBat,runsScoredAfter
1069857,20050826TBDLAA0,LAA,t1,2,123,0,2,2
1420042,20070805NYYKCR0,NYY,b2,2,1-3,4,4,0
579949,20030424KCRMIN0,KCR,b1,0,---,0,0,0
2352844,20120819ATLLAD0,LAD,t5,1,1--,1,1,0
1666022,20090407LAAOAK0,LAA,b1,1,--3,0,1,1
1047233,20050804NYMMIL0,MIL,t6,1,---,6,6,0
2394250,20120928SDPSFG0,SFG,t7,0,---,3,3,0
3439930,20180807TEXSEA0,SEA,t1,1,---,0,0,0
3199515,20170611CHCCOL0,COL,t5,0,12-,0,4,4
3250047,20170802PITCIN0,CIN,t7,2,---,3,3,0


In [80]:
# df[(df['runnerOnBase']=='123') & (df['gameKey']=='20140503BOSOAK0')]

In [81]:
## Group by out and runnerOnBase and compute the average runs score after every situation

dfGb = df.groupby(['out','runnerOnBase']).agg({'runsScoredAfter':'mean'}).reset_index()

In [84]:
## pivot the table so it is easier to read

table = pd.pivot_table(dfGb, values='runsScoredAfter', index=['runnerOnBase'], columns=['out'], aggfunc=np.sum)

In [85]:
# final output
table.sort_index()

out,0,1,2
runnerOnBase,,,
---,0.499760,0.266562,0.102436
--3,1.397855,0.952112,0.361617
-2-,1.122042,0.672857,0.317516
-23,1.982777,1.377490,0.558382
1--,0.876509,0.521011,0.224797
1-3,1.771227,1.151721,0.485202
12-,1.461525,0.901027,0.426501
123,2.257803,1.516798,0.729658
